##### using local conda environment ``keras_augtest``

In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import random
import os

Using TensorFlow backend.


###### I need to come up with a batched approach, i.e. applying one form of augmentation at a time and testing just that augmentation. Then, we can see more specifically how each of these augmentations affects the phash.

**Remember the goal here: Determine expected Hamming distance when rotated, or different background, etc.**

Would be nice to have dictionary from original image to rotations, which is a set of the rotated images containing the hamming distance in phash and in dhash from the original. So should be thinking about structure... how should I code that up...

### TODO: in keras_augtest make cval a 3-tuple, where each value is value to augment for that channel!

In [2]:
def gen_images(datagen, image, number, directory, prefix):
    """@param datagen: a keras ImageDataGenerator class
        @param image: the image of which we'll produce augmentations
        @param number: number of augmentations to save
        @param directory: where to save the augmentations
                            N.B. path must already exist!
        @param prefix: prefix for saving augmented images
                            N.B. does NOT take into account image name"""
    print('Generating augmented images to {} with prefix {}\n...'.format(directory, prefix))
    img = load_img(image)
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  #  is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=directory,
                              save_prefix=prefix,
                              save_format='jpeg'):
        i += 1
        if i > number:
            break  # otherwise the generator would loop indefinitely
    print('Done generating {} images to {}'.format(number, directory))


In [3]:
def gen_cval():
    """Generate a random cval 3-tuple per our udpated implementation that
        will augment using any color as background, not just greyscale.
        Returns: this 3-tuple"""
    return (random.uniform(0., 255.),random.uniform(0., 255.),random.uniform(0., 255.))

In [4]:
def gen_rotation(max_range, cval=gen_cval()):
    """Generate a rotation ImageDataGenerator with given max_range to
        rotate and optional cval 3-tuple.
        If no cval 3-tuple provided, then generates random one
        Returns: the ImageDataGenerator"""
    return ImageDataGenerator(
        rotation_range=max_range,
        fill_mode='constant',
        cval=cval
        )

In [5]:
def gen_zoom(zoom_range, cval=gen_cval()):
    """Generate a zoom ImageDataGenerator with given zoom_range to
        zoom and optional cval 3-tuple.
        If no cval 3-tuple provided, then generates random one
        Returns: the ImageDataGenerator"""
    return ImageDataGenerator(
        zoom_range=zoom_range,
        fill_mode='constant',
        cval=cval
        )

In [6]:
cvals = [gen_cval() for k in range(20)] # generate 20 cval tuples to test
cvals.append((0.,0.,0.)) # add black
cvals.append((255.,255.,255.)) # add white

In [7]:
# main directory within which storing all images
DIR = '../rotation_generation/preview/'
# picture for testing
PIC = '../rotation_generation/ccg.jpg'

In [8]:
# cval0 = gen_cval()

In [9]:
max_range=180

In [10]:
zoom_range = [.33, 3.]

In [11]:
# cval = gen_cval()

In [12]:
# cval = gen_cval()
# gen_images(gen_rotation(max_range, cval=cval), PIC, 50, DIR, 'gen_test')

In [13]:
# cval = gen_cval()
# gen_images(gen_zoom(zoom_range, cval=cval), PIC, 50, DIR, 'gen_zoom_test')

In [14]:
def make_img_list(folder):
    """Returns: list of all files within given folder"""
    return [os.path.join(folder,f).lower() for f in os.listdir(folder)
          if not os.path.isdir(os.path.join(folder,f))]

In [15]:
def core_list(img_list, base):
    """Requires: all image names (i.e. no path and no '.jpg' at end)
        in img_list that have '.jpg' suffix"""
    return [i[len(base):-4] for i in img_list]

In [16]:
DIRECTORY = '../rotation_generation/'

In [17]:
# get list of all qcard names
qcards = core_list(make_img_list(DIRECTORY), DIRECTORY)

In [18]:
qcards

['akpsi_buns',
 'akpsi_recruit',
 'akpsi_women',
 'andrea',
 'card',
 'ccc',
 'ccg',
 'ccg_pwc',
 'deloitte',
 'gabe',
 'paulina',
 'sahil']

In [19]:
for card in qcards:
    # make directories for each of these qcard base names
    os.mkdir(DIRECTORY+card)

In [20]:
# list of paths to all images we're augmenting
qcard_dirs = make_img_list(DIRECTORY)

In [21]:
qcard_dirs

['../rotation_generation/akpsi_buns.jpg',
 '../rotation_generation/akpsi_recruit.jpg',
 '../rotation_generation/akpsi_women.jpg',
 '../rotation_generation/andrea.jpg',
 '../rotation_generation/card.jpg',
 '../rotation_generation/ccc.jpg',
 '../rotation_generation/ccg.jpg',
 '../rotation_generation/ccg_pwc.jpg',
 '../rotation_generation/deloitte.jpg',
 '../rotation_generation/gabe.jpg',
 '../rotation_generation/paulina.jpg',
 '../rotation_generation/sahil.jpg']

In [22]:
# make all the directories where to store the augmented images

for card in qcard_dirs: # iterate over paths to target images
    for k in range(22): # 20 + black + white
        try:
            # make rote and zoom directories for given k and qcard base
            os.mkdir(os.path.join(card[:-4], 'rote_cval'+str(k)))
            os.mkdir(os.path.join(card[:-4], 'zoom_cval'+str(k)))
        except FileExistsError as e:
            # if already made that directory, then don't make it again
            continue
print('Done generating all subdirectories.')

Done generating all subdirectories.


In [23]:
# generate all the augmented images

for card in qcard_dirs: # iterate over paths to target images
    for k in range(22): # 20 + black + white
        cval = cvals[k] # get corresponding cval 3-tuple
        gen_images(gen_rotation(max_range, cval=cval), # generate rotations
                   card, 50, card[:-4]+'/rote_cval'+str(k),
                   'rote_cval'+str(k))
        gen_images(gen_zoom(zoom_range, cval=cval), # generate zooms
                   card, 50, card[:-4]+'/zoom_cval'+str(k),
                   'zoom_cval'+str(k))

Generating augmented images to ../rotation_generation/akpsi_buns/rote_cval0 with prefix rote_cval0
...
Done generating 50 images to ../rotation_generation/akpsi_buns/rote_cval0
Generating augmented images to ../rotation_generation/akpsi_buns/zoom_cval0 with prefix zoom_cval0
...
Done generating 50 images to ../rotation_generation/akpsi_buns/zoom_cval0
Generating augmented images to ../rotation_generation/akpsi_buns/rote_cval1 with prefix rote_cval1
...
Done generating 50 images to ../rotation_generation/akpsi_buns/rote_cval1
Generating augmented images to ../rotation_generation/akpsi_buns/zoom_cval1 with prefix zoom_cval1
...
Done generating 50 images to ../rotation_generation/akpsi_buns/zoom_cval1
Generating augmented images to ../rotation_generation/akpsi_buns/rote_cval2 with prefix rote_cval2
...
Done generating 50 images to ../rotation_generation/akpsi_buns/rote_cval2
Generating augmented images to ../rotation_generation/akpsi_buns/zoom_cval2 with prefix zoom_cval2
...
Done generat

Done generating 50 images to ../rotation_generation/akpsi_recruit/rote_cval1
Generating augmented images to ../rotation_generation/akpsi_recruit/zoom_cval1 with prefix zoom_cval1
...
Done generating 50 images to ../rotation_generation/akpsi_recruit/zoom_cval1
Generating augmented images to ../rotation_generation/akpsi_recruit/rote_cval2 with prefix rote_cval2
...
Done generating 50 images to ../rotation_generation/akpsi_recruit/rote_cval2
Generating augmented images to ../rotation_generation/akpsi_recruit/zoom_cval2 with prefix zoom_cval2
...
Done generating 50 images to ../rotation_generation/akpsi_recruit/zoom_cval2
Generating augmented images to ../rotation_generation/akpsi_recruit/rote_cval3 with prefix rote_cval3
...
Done generating 50 images to ../rotation_generation/akpsi_recruit/rote_cval3
Generating augmented images to ../rotation_generation/akpsi_recruit/zoom_cval3 with prefix zoom_cval3
...
Done generating 50 images to ../rotation_generation/akpsi_recruit/zoom_cval3
Generati

Done generating 50 images to ../rotation_generation/akpsi_women/zoom_cval1
Generating augmented images to ../rotation_generation/akpsi_women/rote_cval2 with prefix rote_cval2
...
Done generating 50 images to ../rotation_generation/akpsi_women/rote_cval2
Generating augmented images to ../rotation_generation/akpsi_women/zoom_cval2 with prefix zoom_cval2
...
Done generating 50 images to ../rotation_generation/akpsi_women/zoom_cval2
Generating augmented images to ../rotation_generation/akpsi_women/rote_cval3 with prefix rote_cval3
...
Done generating 50 images to ../rotation_generation/akpsi_women/rote_cval3
Generating augmented images to ../rotation_generation/akpsi_women/zoom_cval3 with prefix zoom_cval3
...
Done generating 50 images to ../rotation_generation/akpsi_women/zoom_cval3
Generating augmented images to ../rotation_generation/akpsi_women/rote_cval4 with prefix rote_cval4
...
Done generating 50 images to ../rotation_generation/akpsi_women/rote_cval4
Generating augmented images to

Done generating 50 images to ../rotation_generation/andrea/zoom_cval2
Generating augmented images to ../rotation_generation/andrea/rote_cval3 with prefix rote_cval3
...
Done generating 50 images to ../rotation_generation/andrea/rote_cval3
Generating augmented images to ../rotation_generation/andrea/zoom_cval3 with prefix zoom_cval3
...
Done generating 50 images to ../rotation_generation/andrea/zoom_cval3
Generating augmented images to ../rotation_generation/andrea/rote_cval4 with prefix rote_cval4
...
Done generating 50 images to ../rotation_generation/andrea/rote_cval4
Generating augmented images to ../rotation_generation/andrea/zoom_cval4 with prefix zoom_cval4
...
Done generating 50 images to ../rotation_generation/andrea/zoom_cval4
Generating augmented images to ../rotation_generation/andrea/rote_cval5 with prefix rote_cval5
...
Done generating 50 images to ../rotation_generation/andrea/rote_cval5
Generating augmented images to ../rotation_generation/andrea/zoom_cval5 with prefix z

Done generating 50 images to ../rotation_generation/card/rote_cval5
Generating augmented images to ../rotation_generation/card/zoom_cval5 with prefix zoom_cval5
...
Done generating 50 images to ../rotation_generation/card/zoom_cval5
Generating augmented images to ../rotation_generation/card/rote_cval6 with prefix rote_cval6
...
Done generating 50 images to ../rotation_generation/card/rote_cval6
Generating augmented images to ../rotation_generation/card/zoom_cval6 with prefix zoom_cval6
...
Done generating 50 images to ../rotation_generation/card/zoom_cval6
Generating augmented images to ../rotation_generation/card/rote_cval7 with prefix rote_cval7
...
Done generating 50 images to ../rotation_generation/card/rote_cval7
Generating augmented images to ../rotation_generation/card/zoom_cval7 with prefix zoom_cval7
...
Done generating 50 images to ../rotation_generation/card/zoom_cval7
Generating augmented images to ../rotation_generation/card/rote_cval8 with prefix rote_cval8
...
Done gener

...

Done generating 50 images to ../rotation_generation/ccg/rote_cval11
Generating augmented images to ../rotation_generation/ccg/zoom_cval11 with prefix zoom_cval11
...
Done generating 50 images to ../rotation_generation/ccg/zoom_cval11
Generating augmented images to ../rotation_generation/ccg/rote_cval12 with prefix rote_cval12
...
Done generating 50 images to ../rotation_generation/ccg/rote_cval12
Generating augmented images to ../rotation_generation/ccg/zoom_cval12 with prefix zoom_cval12
...
Done generating 50 images to ../rotation_generation/ccg/zoom_cval12
Generating augmented images to ../rotation_generation/ccg/rote_cval13 with prefix rote_cval13
...
Done generating 50 images to ../rotation_generation/ccg/rote_cval13
Generating augmented images to ../rotation_generation/ccg/zoom_cval13 with prefix zoom_cval13
...
Done generating 50 images to ../rotation_generation/ccg/zoom_cval13
Generating augmented images to ../rotation_generation/ccg/rote_cval14 with prefix rote_cval14
...
Done

Done generating 50 images to ../rotation_generation/ccg_pwc/zoom_cval13
Generating augmented images to ../rotation_generation/ccg_pwc/rote_cval14 with prefix rote_cval14
...
Done generating 50 images to ../rotation_generation/ccg_pwc/rote_cval14
Generating augmented images to ../rotation_generation/ccg_pwc/zoom_cval14 with prefix zoom_cval14
...
Done generating 50 images to ../rotation_generation/ccg_pwc/zoom_cval14
Generating augmented images to ../rotation_generation/ccg_pwc/rote_cval15 with prefix rote_cval15
...
Done generating 50 images to ../rotation_generation/ccg_pwc/rote_cval15
Generating augmented images to ../rotation_generation/ccg_pwc/zoom_cval15 with prefix zoom_cval15
...
Done generating 50 images to ../rotation_generation/ccg_pwc/zoom_cval15
Generating augmented images to ../rotation_generation/ccg_pwc/rote_cval16 with prefix rote_cval16
...
Done generating 50 images to ../rotation_generation/ccg_pwc/rote_cval16
Generating augmented images to ../rotation_generation/ccg_

Done generating 50 images to ../rotation_generation/deloitte/zoom_cval15
Generating augmented images to ../rotation_generation/deloitte/rote_cval16 with prefix rote_cval16
...
Done generating 50 images to ../rotation_generation/deloitte/rote_cval16
Generating augmented images to ../rotation_generation/deloitte/zoom_cval16 with prefix zoom_cval16
...
Done generating 50 images to ../rotation_generation/deloitte/zoom_cval16
Generating augmented images to ../rotation_generation/deloitte/rote_cval17 with prefix rote_cval17
...
Done generating 50 images to ../rotation_generation/deloitte/rote_cval17
Generating augmented images to ../rotation_generation/deloitte/zoom_cval17 with prefix zoom_cval17
...
Done generating 50 images to ../rotation_generation/deloitte/zoom_cval17
Generating augmented images to ../rotation_generation/deloitte/rote_cval18 with prefix rote_cval18
...
Done generating 50 images to ../rotation_generation/deloitte/rote_cval18
Generating augmented images to ../rotation_gene

Done generating 50 images to ../rotation_generation/gabe/rote_cval18
Generating augmented images to ../rotation_generation/gabe/zoom_cval18 with prefix zoom_cval18
...
Done generating 50 images to ../rotation_generation/gabe/zoom_cval18
Generating augmented images to ../rotation_generation/gabe/rote_cval19 with prefix rote_cval19
...
Done generating 50 images to ../rotation_generation/gabe/rote_cval19
Generating augmented images to ../rotation_generation/gabe/zoom_cval19 with prefix zoom_cval19
...
Done generating 50 images to ../rotation_generation/gabe/zoom_cval19
Generating augmented images to ../rotation_generation/gabe/rote_cval20 with prefix rote_cval20
...
Done generating 50 images to ../rotation_generation/gabe/rote_cval20
Generating augmented images to ../rotation_generation/gabe/zoom_cval20 with prefix zoom_cval20
...
Done generating 50 images to ../rotation_generation/gabe/zoom_cval20
Generating augmented images to ../rotation_generation/gabe/rote_cval21 with prefix rote_cva

Done generating 50 images to ../rotation_generation/paulina/rote_cval20
Generating augmented images to ../rotation_generation/paulina/zoom_cval20 with prefix zoom_cval20
...
Done generating 50 images to ../rotation_generation/paulina/zoom_cval20
Generating augmented images to ../rotation_generation/paulina/rote_cval21 with prefix rote_cval21
...
Done generating 50 images to ../rotation_generation/paulina/rote_cval21
Generating augmented images to ../rotation_generation/paulina/zoom_cval21 with prefix zoom_cval21
...
Done generating 50 images to ../rotation_generation/paulina/zoom_cval21
Generating augmented images to ../rotation_generation/sahil/rote_cval0 with prefix rote_cval0
...
Done generating 50 images to ../rotation_generation/sahil/rote_cval0
Generating augmented images to ../rotation_generation/sahil/zoom_cval0 with prefix zoom_cval0
...
Done generating 50 images to ../rotation_generation/sahil/zoom_cval0
Generating augmented images to ../rotation_generation/sahil/rote_cval1 w

In [24]:
# datagen = ImageDataGenerator(
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest',
#         cval=(0.,0.,0.))

# img = load_img('../rotation_generation/ccg.jpg')
# x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1,
#                           save_to_dir='../rotation_generation/preview/', save_prefix='ccg_blog', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely


In [25]:
# datagen_rot_cval_tup = ImageDataGenerator(
#         rotation_range=40,
#         fill_mode='constant',
#         cval=(30.,100.,255.)
#         )

In [26]:
# img = load_img('../rotation_generation/ccg.jpg')
# x = img_to_array(img)
# x = x.reshape((1,) + x.shape)

In [27]:
# i = 0
# for batch in datagen_rot_cval_tup.flow(x, batch_size=1,
#                           save_to_dir='../rotation_generation/preview/', save_prefix='ccg_rot_cval_tup', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely


In [28]:
# datagen_zoom = ImageDataGenerator(
#         zoom_range=[1.,5.],
#         fill_mode='constant',
#         cval=(0.,0.,0.)
#         )

In [29]:
# # img = load_img('../rotation_generation/ccg.jpg')
# x = img_to_array(img)
# x = x.reshape((1,) + x.shape)

In [30]:
# i = 0
# for batch in datagen_zoom.flow(x, batch_size=1,
#                           save_to_dir='../rotation_generation/preview/',
#                                save_prefix='ccg_zoom',
#                                save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely


## happening 'cuz should make another apply_transform function with cval as tuple for rotations only

In [31]:
# gen_images(datagen_zoom, '../rotation_generation/ccg.jpg', 
#           50, '../rotation_generation/preview/', 'func_test')